<a href="https://colab.research.google.com/github/Udhayakumarbalasubramaniam/AI-MOUSE-/blob/main/FAKE_NEWS_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import re
import string


In [ ]:
# Read data
data_fake = pd.read_csv('/content/Fake.csv')
data_true = pd.read_csv('/content/True.csv')


In [ ]:
data_fake["class"] = 0
data_true['class'] = 1

In [ ]:
# Drop rows for manual testing
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480, 23470, -1):
    data_fake.drop([i], axis=0, inplace=True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416, 21406, -1):
    data_true.drop([i], axis=0, inplace=True)

In [ ]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

<ipython-input-143-90008d39c97b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
<ipython-input-143-90008d39c97b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [ ]:
# Concatenate data
data_merge = pd.concat([data_fake, data_true], axis=0)

data = data_merge.drop(['title', 'subject', 'date'], axis=1)

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\W', '', text)
    text = re.sub(r'https?://\S+/www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

In [ ]:
data['text'] = data['text'].apply(wordopt)

In [ ]:
x = data['text']
y = data['class']

In [ ]:
# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.50)

In [ ]:
# Vectorize the training and testing data
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [ ]:
# Logistic Regression model
LR = LogisticRegression()
LR.fit(xv_train, y_train)
pred_lr = LR.predict(xv_test)

In [ ]:
# Evaluate Logistic Regression model
print("Logistic Regression Model:")
print("Accuracy:", LR.score(xv_test, y_test))
print(classification_report(y_test, pred_lr))

Logistic Regression Model:
Accuracy: 0.5258255715495342
              precision    recall  f1-score   support

           0       0.52      1.00      0.69     11737
           1       1.00      0.01      0.01     10702

    accuracy                           0.53     22439
   macro avg       0.76      0.50      0.35     22439
weighted avg       0.75      0.53      0.37     22439



In [ ]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)
pred_dt = DT.predict(xv_test)

In [ ]:
print("\nDecision Tree Model:")
print("Accuracy:", DT.score(xv_test, y_test))
print(classification_report(y_test, pred_dt))


Decision Tree Model:
Accuracy: 0.5258255715495342
              precision    recall  f1-score   support

           0       0.52      1.00      0.69     11737
           1       1.00      0.01      0.01     10702

    accuracy                           0.53     22439
   macro avg       0.76      0.50      0.35     22439
weighted avg       0.75      0.53      0.37     22439



In [ ]:
# Manual testing function
def output_label(n):
    if n == 0:
        return "aFake News"
    elif n == 1:
        return "Not A Fake News"


In [ ]:
def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)

    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)

    # Predictions
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)

    # Output results
    print("\n\nLR Prediction: {}\nDT Prediction: {}".format(output_label(pred_LR[0]), output_label(pred_DT[0])))

In [ ]:
# Manual testing input
news_input ="ljuyljhyxkuhgdx "
manual_testing(news_input)



LR Prediction: aFake News
DT Prediction: aFake News
